<a href="https://colab.research.google.com/github/maxi-gkd/sales-forecast/blob/main/WeeklyForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fbprophet

**1. Import Dependencies**

In [2]:
import pandas as pd
import numpy as np
from math import ceil

from fbprophet import Prophet
from fbprophet.plot import plot_plotly

from google.colab import drive 

**2. Load Data**


In [ ]:
drive.mount('/content/gdrive')


In [4]:
df = pd.read_csv('gdrive/MyDrive/TP_gestion_comercial/weekly_sales_from_2017.csv')

**3. Apply Data Preprocessing**


In [5]:
df["qty"] = 1
df['ds'] = pd.to_datetime(df['ds'])
df.drop('total', axis=1, inplace=True)

In [6]:
df_weekly = df.resample('W', on='ds')['qty'].sum()
df_weekly = df_weekly.reset_index()

In [7]:
df_weekly.columns = ['ds', 'y']

**4. Create Time Series Model**


In [8]:
m = Prophet(daily_seasonality = False, weekly_seasonality = False, yearly_seasonality = True, interval_width=0.80) 


In [9]:
training_run = m.fit(df_weekly)


**5. Evaluate Model**


In [10]:
future = m.make_future_dataframe(periods=24, freq='W')


In [11]:
forecast = m.predict(future)


In [12]:
result = forecast[['ds','yhat', 'yhat_lower', 'yhat_upper']].copy()
result['yhat'] = result['yhat'].astype(np.int64)
result['yhat_lower'] = result['yhat_lower'].astype(np.int64)
result['yhat_upper'] = result['yhat_upper'].astype(np.int64)

In [ ]:
result.tail(20)

In [ ]:
fig=plot_plotly(m, forecast)
fig

In [ ]:
plot2 = m.plot_components(forecast)


**6. Process & Show results**

In [38]:
#group sales by quarter
monthly_result = result.groupby(result['ds'].dt.to_period('Q')).agg('sum')
monthly_result.drop('yhat_lower', axis=1, inplace=True)
monthly_result.drop('yhat_upper', axis=1, inplace=True)
monthly_result.columns = ['Flights sold']

In [ ]:
monthly_result.tail(10)